In [ ]:
import glob
import pandas as pd
import os
import torch
import librosa
import random

from BEATs.Tokenizers import TokenizersConfig, Tokenizers
from BEATs.BEATs import BEATs, BEATsConfig

In [ ]:
data_folder = "/data/ESC-50-master"
audio = glob.glob(data_folder + "/audio/*.wav", recursive=True)
labels = pd.read_csv(os.path.join(data_folder, "meta/esc50.csv"))

In [ ]:
df_audio = pd.DataFrame(audio, columns=["filepath"])
df_audio["filename"] = [f.split("/")[-1] for f in audio]
filepath_labels = labels.merge(df_audio, how="inner", on="filename")

In [ ]:
checkpoint = torch.load('/data/BEATs/BEATs_iter3_plus_AS2M.pt')

cfg = BEATsConfig({
    **checkpoint['cfg'],
    "predictor_class": 50,
    "finetuned_model": False
})
        
#cfg = BEATsConfig(checkpoint['cfg'])
BEATs_model = BEATs(cfg)
BEATs_model.load_state_dict(checkpoint['model'])
BEATs_model.eval()

In [ ]:
# load the pre-trained checkpoints
checkpoint = torch.load('/data/BEATs/BEATs_iter3_plus_AS2M.pt')
cfg = BEATsConfig(checkpoint['cfg'])
BEATs_model = BEATs(cfg)
BEATs_model.load_state_dict(checkpoint['model'])

In [ ]:
import torch.nn as nn

In [ ]:
_layers = list(BEATs_model.children())[:-1]
feature_extractor = nn.Sequential(*_layers)

In [ ]:
feature_extractor